# SparkSQL basics with SparkR  

[**Introduction to Apache Spark with R by J. A. Dianes**](https://github.com/jadianes/spark-r-notebooks)

In this notebook we will introduce basic concepts about SparkSQL with R that you can find in the [SparkR documentation](http://spark.apache.org/docs/latest/sparkr.html), applied to the [2013 American Community Survey dataset](http://www.census.gov/programs-surveys/acs/data/summary-file.html). We will do two things, read data into a SparkSQL data frame, and have a quick look at the schema and what we have read. 

## Creating a SparkSQL context

In further notebooks, we will explore our data by loading them into SparkSQL data frames. But first we need to init a SparkSQL context. The first thing we need to do is to set up some environment variables and library paths as follows. Remember to replace the value assigned to `SPARK_HOME` with your Spark home folder.  

In [1]:
# Set Spark home and R libs
# Sys.setenv(SPARK_HOME='/home/cluster/spark-1.5.0-bin-hadoop2.6')
# .libPaths(c(file.path(Sys.getenv('SPARK_HOME'), 'R', 'lib'), .libPaths()))

Now we can load the `SparkR` library as follows.

In [1]:
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window

The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union



And now we can initialise the Spark context as [in the official documentation](http://spark.apache.org/docs/latest/sparkr.html#starting-up-sparkcontext-sqlcontext). In our case we are use a standalone Spark cluster with one master and seven workers. If you are running Spark in local node, use just `master='local'`. Additionally, we require a Spark package from Databricks to read CSV files (more on this in the next section).  

In [2]:
# sc <- sparkR.init(master='spark://169.254.206.2:7077', sparkPackages="com.databricks:spark-csv_2.11:1.2.0")
sqlContext <- sparkR.session(master = "local[*]", sparkConfig = list(spark.driver.memory = "2g") ,sparkPackages = "com.databricks:spark-csv_2.11:1.2.0")


Spark package found in SPARK_HOME: /opt/spark


Launching java with spark-submit command /opt/spark/bin/spark-submit  --packages com.databricks:spark-csv_2.11:1.2.0 --driver-memory "2g" sparkr-shell /tmp/RtmpqMAnJW/backend_port27014aef5147 


And finally we can start the SparkSQL context as follows.

In [3]:
#sqlContext <- sparkRSQL.init(sc)

## Creating SparkSQL data frames

### Reading CSV data using Databricks csv extension

The easiest way to get our CSV data into a [SparkSQL dataframe](http://spark.apache.org/docs/latest/sparkr.html#creating-dataframes), is by using [Databricks CSV extension](https://github.com/databricks/spark-csv) to read SparkSQL dataframes directly from csv files. In any case, remember to set the right path for your data files in the first line, ours is `/home/spark/pu_workshop/data/2013-acs/ss13husa.csv`.  

In [4]:
housing_a_file_path <- file.path('', 'home','spark','pu_workshop','data','2013-acs','ss13husa.csv')
housing_b_file_path <- file.path('', 'home','spark','pu_workshop','data','2013-acs','ss13husa.csv')

Now let's read into a SparkSQL dataframe. We need to pass four parameters in addition to the `sqlContext`:  

- The file path.  
- `header='true'` since our `csv` files have a header with the column names. 
- Indicate that we want the library to infer the schema.  
- And the source type (the Databricks package in this case).  

In [5]:
system.time(
    housing_a_df <- read.df(
        #sqlContext, 
                        housing_a_file_path, 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')
)

   user  system elapsed 
  0.004   0.000  19.726 

Let's have a look at the inferred schema.

In [6]:
system.time(
    printSchema(housing_a_df)
)

root
 |-- RT: string (nullable = true)
 |-- SERIALNO: integer (nullable = true)
 |-- DIVISION: integer (nullable = true)
 |-- PUMA: integer (nullable = true)
 |-- REGION: integer (nullable = true)
 |-- ST: integer (nullable = true)
 |-- ADJHSG: integer (nullable = true)
 |-- ADJINC: integer (nullable = true)
 |-- WGTP: integer (nullable = true)
 |-- NP: integer (nullable = true)
 |-- TYPE: integer (nullable = true)
 |-- ACCESS: integer (nullable = true)
 |-- ACR: integer (nullable = true)
 |-- AGS: integer (nullable = true)
 |-- BATH: integer (nullable = true)
 |-- BDSP: integer (nullable = true)
 |-- BLD: integer (nullable = true)
 |-- BROADBND: integer (nullable = true)
 |-- BUS: integer (nullable = true)
 |-- COMPOTHX: integer (nullable = true)
 |-- CONP: integer (nullable = true)
 |-- DIALUP: integer (nullable = true)
 |-- DSL: integer (nullable = true)
 |-- ELEP: integer (nullable = true)
 |-- FIBEROP: integer (nullable = true)
 |-- FS: integer (nullable = true)
 |-- FULP: integer

   user  system elapsed 
  0.000   0.004   0.060 

Looks good. Let's have a look at the first few rows.  

In [7]:
head(housing_a_df)

RT,SERIALNO,DIVISION,PUMA,REGION,ST,ADJHSG,ADJINC,WGTP,NP,⋯,wgtp71,wgtp72,wgtp73,wgtp74,wgtp75,wgtp76,wgtp77,wgtp78,wgtp79,wgtp80
H,84,6,2600,3,1,1000000,1007549,0,1,⋯,0,0,0,0,0,0,0,0,0,0
H,154,6,2500,3,1,1000000,1007549,51,4,⋯,86,53,59,84,49,15,15,20,50,16
H,156,6,1700,3,1,1000000,1007549,449,1,⋯,161,530,601,579,341,378,387,421,621,486
H,160,6,2200,3,1,1000000,1007549,16,3,⋯,31,24,33,7,7,13,18,23,23,5
H,231,6,2400,3,1,1000000,1007549,52,1,⋯,21,18,37,49,103,38,49,51,46,47
H,286,6,900,3,1,1000000,1007549,76,1,⋯,128,25,68,66,80,26,66,164,88,24


And let's count how many rows do we have in the first dataset. For that we will use `nrow` as we do with regular R data frames. Let's have a look at the documentation.  

In [8]:
?nrow

It is just a definition of `nrow` by the package *SparkR*.

In [9]:
nrow(housing_a_df)

[1] 756065

Let's read the second housing data frame and count the number of rows.

In [10]:
system.time(
    housing_b_df <- read.df(
        #sqlContext, 
                        housing_b_file_path, 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')
)

   user  system elapsed 
  0.021   0.094  15.259 

In [11]:
print(nrow(housing_b_df))

[1] 756065


### Merging data frames

Now we can use `rbind()` as we do with regular R data frames to put both of them together. Let's have a look at the documentation.  

In [12]:
?rbind

Again, SparkR redefines many of the common R functions to work with SparkSQL data frames. Let's actually use `rbind` as follows.

In [13]:
housing_df <- rbind(housing_a_df, housing_b_df)

And let's count how many rows do we have in the complete data frame.

In [14]:
system.time(
    housing_samples <- nrow(housing_df)
)
print(housing_samples)

   user  system elapsed 
  0.001   0.000  10.245 

[1] 1512130


Finally, let's get a feeling of what is to explore data using SparkR by using the `summary` function on the data frame. Let's first have a look at the documentation.

In [15]:
? summary

We can see that we are redirected to `describe`, that optionally accepts column names. Let's use it with the whole data frame, that is, the 231 columns and 1476313 rows. *Note*: We use `collect` here because the results of `describe` are given as a `DataFrame` object and we need to print them in the notebook.

In [ ]:
system.time(
    housing_summary <- describe(housing_df)
)

In [ ]:
collect(housing_summary)

Or we can select individual column summaries using `select` as follows (here is [a dictionary](http://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict13.txt) of each column meaning) where `VALP` is the property value.  

In [25]:
collect(select(housing_summary,"VALP"))

,VALP
1,859691
2,247682.84302150423
3,NaN
4,100
5,4775000


And that's it. In this notebook we have shown how to load a CSV file into an SparkSQL data frame using SparkR. We also had a look at the data loaded, mainly to the number of samples loaded, and the data summary.